In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [21]:
fuel_cons_2023 = pd.read_csv("https://www.nrcan.gc.ca/sites/nrcan/files/oee/files/csv/MY2023%20Fuel%20Consumption%20Ratings.csv", encoding="Windows-1252")
fuel_cons_2022 = pd.read_csv("https://www.nrcan.gc.ca/sites/nrcan/files/oee/files/csv/MY2022%20Fuel%20Consumption%20Ratings.csv", encoding="Windows-1252")
fuel_cons_2021 = pd.read_csv("https://www.nrcan.gc.ca/sites/nrcan/files/oee/files/csv/MY2021%20Fuel%20Consumption%20Ratings.csv", encoding="Windows-1252")
fuel_cons_2020 = pd.read_csv("https://www.nrcan.gc.ca/sites/nrcan/files/oee/files/csv/MY2020%20Fuel%20Consumption%20Ratings.csv", encoding="Windows-1252")
fuel_cons_2019 = pd.read_csv("https://www.nrcan.gc.ca/sites/nrcan/files/oee/files/csv/MY2019%20Fuel%20Consumption%20Ratings.csv", encoding="Windows-1252")
fuel_cons_2018 = pd.read_csv("https://www.nrcan.gc.ca/sites/nrcan/files/oee/files/csv/MY2018%20Fuel%20Consumption%20Ratings.csv", encoding="Windows-1252")
fuel_cons_2017 = pd.read_csv("https://www.nrcan.gc.ca/sites/nrcan/files/oee/files/csv/MY2017%20Fuel%20Consumption%20Ratings.csv", encoding="Windows-1252")
fuel_cons_2016 = pd.read_csv("https://www.nrcan.gc.ca/sites/nrcan/files/oee/files/csv/MY2016%20Fuel%20Consumption%20Ratings.csv", encoding="Windows-1252")
fuel_cons_2015 = pd.read_csv("https://www.nrcan.gc.ca/sites/nrcan/files/oee/files/csv/MY2015%20Fuel%20Consumption%20Ratings.csv", encoding="Windows-1252")
fuel_cons_2010_2014 = pd.read_csv("https://www.nrcan.gc.ca/sites/www.rncan.gc.ca/files/oee/files/csv/MY2010-2014%20Fuel%20Consumption%20Ratings%205-cycle.csv", encoding="Windows-1252")
fuel_cons_2005_2009 = pd.read_csv("https://www.nrcan.gc.ca/sites/www.rncan.gc.ca/files/oee/files/csv/MY2005-2009%20Fuel%20Consumption%20Ratings%205-cycle.csv", encoding="Windows-1252")
fuel_cons_2000_2004 = pd.read_csv("https://www.nrcan.gc.ca/sites/www.rncan.gc.ca/files/oee/files/csv/MY2000-2004%20Fuel%20Consumption%20Ratings%205-cycle.csv", encoding="Windows-1252")


/var/folders/7c/glh90ckd7bdfn7cc0xbs4hgr0000gn/T/ipykernel_35397/215738493.py:1: DtypeWarning: Columns (0,1,2,3,4,6,7,8,9,10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  fuel_cons_2023 = pd.read_csv("https://www.nrcan.gc.ca/sites/nrcan/files/oee/files/csv/MY2023%20Fuel%20Consumption%20Ratings.csv", encoding="Windows-1252")
/var/folders/7c/glh90ckd7bdfn7cc0xbs4hgr0000gn/T/ipykernel_35397/215738493.py:2: DtypeWarning: Columns (0,1,2,3,4,6,7,8,9,10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  fuel_cons_2022 = pd.read_csv("https://www.nrcan.gc.ca/sites/nrcan/files/oee/files/csv/MY2022%20Fuel%20Consumption%20Ratings.csv", encoding="Windows-1252")
/var/folders/7c/glh90ckd7bdfn7cc0xbs4hgr0000gn/T/ipykernel_35397/215738493.py:3: DtypeWarning: Columns (0,1,2,3,4,6,7,8,9,10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  fuel_cons_2021 = pd.read_csv("https://www.nrcan.